In [1]:
# import gdown ; 
import pandas as pd ; 
import numpy as np;
import math
import os
# train_log_url = "https://drive.google.com/file/d/1VebJgr8LlRWeCsAnYA2OK6VzdtsB9L83/view?usp=drive_link"
# gdown.download(url=train_log_url, quiet=False, fuzzy=True)

# train_score_url = "https://drive.google.com/file/d/1VQ05dODI0aKyvu2RB0-WU2hzk4_bqJyw/view?usp=sharing"
# gdown.download(url=train_score_url, quiet=False, fuzzy=True)

In [2]:
##function to apply on each row for train_logs
def pause_time_to_string(value):
    ## If the pause is bigger than 1000(1sec), value/1000 that many pause to column
    if math.isnan(value)or value <= 1000 :
        return ""
    else:
        sec = round(value/ 1000)
        return " ".join(['pause' for i in range(sec)])

In [3]:
def combine_columns(df):
    if df['pause_str'] != '':
        return df['down_event'] + " " + df['pause_str'] 
    else:
        return df['down_event']

In [4]:
##function to generate and return a string of hmm sequence array
def genHMMInput(df):
    df = df.copy()
    ##create a new coulmn that contains pause time
    df["next_down_time"] = df["down_time"].shift(-1)
    df["pause_time"] = df["next_down_time"] - df["down_time"]
    ####create new column pause_str.
    df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
    df['hmm_input'] = df.apply(combine_columns, axis=1)
    mask = df['id'] != df['id'].shift(-1)
    mask.iloc[-1] = False
    df.loc[mask, 'hmm_input'] += " K"
    hmm_input = " ".join(df["hmm_input"].tolist())
    return hmm_input

In [5]:
# # df2 = pd.read_csv("train_logs.csv")
# df = df2.loc[0:4].copy()
# df["next_down_time"] = df["down_time"].shift(-1)
# df["pause_time"] = df["next_down_time"] - df["down_time"]
# ####create new column pause_str.
# df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
# df['hmm_input'] = df.apply(combine_columns, axis=1)
# mask = df['id'] != df['id'].shift(-1)
# mask.iloc[-1] = False
# df.loc[mask, 'hmm_input'] += " K"
# hmm_input = " ".join(df["hmm_input"].tolist())
# hmm_input

In [6]:
##function that call the above and write them into files
def generateFile(logs, score):
    train_dfs = []
    test_dfs = []
    
    logs_df = pd.read_csv(logs)
    score_df  = pd.read_csv(score)

    if not os.path.exists("train"):
            os.makedirs("train")
    if not os.path.exists("test"):
            os.makedirs("test")
    ##generate inner join dataframe
    log_join_score = pd.merge(logs_df, score_df, on="id", how='inner')
    for s in score_df["score"].unique():
        ##join log and score dataframe base on id
        cur_score_df = log_join_score.loc[log_join_score["score"] == s]
        unique_doc_id = cur_score_df["id"].unique()
        num_unqiue = len(unique_doc_id)
        ##set up randomness and raito for train/test split
        np.random.shuffle(unique_doc_id)
        split_ratio = 0.8
        split_index = int(len(unique_doc_id) * split_ratio)

        ##set the ids that are in train set and test set
        train_id = unique_doc_id[:split_index]
        test_id = unique_doc_id[split_index:]

        len_train = len(train_id)
        len_test = len(test_id)
        print(f"Total of {num_unqiue} for score {s}, train will have {len_train} and test will have {len_test}")

        ##create df for train and test
        train_df = log_join_score[log_join_score['id'].isin(train_id)]
        test_df = log_join_score[log_join_score['id'].isin(test_id)]

        ##add them to the list
        train_dfs.append(train_df)
        test_dfs.append(test_df)

        for df in train_dfs:
            s = str(s)
            hmm_input = genHMMInput(df)
            with open( "train/" +s +".txt", 'w') as file:
                file.write(hmm_input)
        for df in test_dfs:
            hmm_input = genHMMInput(df)
            with open( "test/" +s +".txt", 'w') as file:
                file.write(hmm_input)
    

generateFile("train_logs.csv", "train_scores.csv")

Total of 486 for score 3.5, train will have 388 and test will have 98
Total of 37 for score 6.0, train will have 29 and test will have 8
Total of 92 for score 2.0, train will have 73 and test will have 19
Total of 501 for score 4.0, train will have 400 and test will have 101
Total of 402 for score 4.5, train will have 321 and test will have 81
Total of 201 for score 2.5, train will have 160 and test will have 41
Total of 179 for score 5.0, train will have 143 and test will have 36
Total of 336 for score 3.0, train will have 268 and test will have 68
Total of 69 for score 1.5, train will have 55 and test will have 14
Total of 128 for score 5.5, train will have 102 and test will have 26
Total of 35 for score 1.0, train will have 28 and test will have 7
Total of 5 for score 0.5, train will have 4 and test will have 1


In [14]:
with open("train/3.5.txt", "r") as file:
    print(len(file.readline()))

7402306


In [7]:
# ##write only 3.5 to a csv file
# logs_df = pd.read_csv("train_logs.csv")
# score_df = pd.read_csv("train_scores.csv")
# log_join_score = pd.merge(logs_df, score_df, on="id", how='inner')
# logs_df[log_join_score['score']==3.5].to_csv("35.csv")

In [8]:
# df = pd.read_csv("35.csv")
# df["next_down_time"] = df["down_time"].shift(-1)
# df["pause_time"] = df["next_down_time"] - df["down_time"]
# ####create new column pause_str.
# df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
# df['hmm_input'] = df.apply(combine_columns, axis=1)
# mask = df['id'] != df['id'].shift(-1)
# mask.iloc[-1] = False
# df.loc[mask, 'hmm_input'] += " K"
# hmm_input = " ".join(df["hmm_input"].tolist())

In [9]:
# df[df['hmm_input'].str.contains('Å')]

In [10]:
# column_name = "pause_time"
# # plt.scatter(train_logs.index, train_logs[column_name])

# # # Add labels and title
# # plt.xlabel('Index')
# # plt.ylabel(column_name)
# # plt.title(f'Scatter Plot of {column_name} vs Index')

# # # Show the plot
# # # plt.show()

# percentiles = [25,50,75,80,85] + list(range(90,101))
# # percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [11]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming you already have a DataFrame named 'df'
# # Replace 'column_name' with the actual column you want to plot
# column_name = 'action_time'
# # train_logs["action_time"] = train_logs["action_time"].astype(int)
# mask = (train_logs["action_time"] <=  10000)
# df = train_logs[mask]
# # Create a scatter plot
# plt.scatter(df.index, df[column_name])

# # Add labels and title
# plt.xlabel('Index')
# plt.ylabel(column_name)
# plt.title(f'Scatter Plot of {column_name} vs Index')

# # Show the plot
# plt.show()

# percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [12]:
# # Assuming you already have a DataFrame named 'df'
# # Replace 'column_name' with the actual column you want to plot
# column_name = ''
# # train_logs["action_time"] = train_logs["action_time"].astype(int)
# mask = (train_logs["action_time"] <=  10000)
# df = train_logs[mask]
# # Create a scatter plot
# plt.scatter(df.index, df[column_name])

# # Add labels and title
# plt.xlabel('Index')
# plt.ylabel(column_name)
# plt.title(f'Scatter Plot of {column_name} vs Index')

# # Show the plot
# plt.show()

# percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [13]:
# ##find the gap between down/up time
# print(max(train_logs["action_time"]), min(train_logs["action_time"]))

# for i, r in train_logs.iterrows():
#   cur_event = r["down_time"]